<a href="https://colab.research.google.com/github/CorsiDanilo/big-data-computing-project/blob/main/0_Data_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bitcoin price forecasting with PySpark
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



Description: In this notebook, I will be responsible for retrieving data from www.blockchain.com by considering the most interesting features, generate the dataset containing the minute-by-minute (1m) and daily (1d) data, and save it to the Google Drive space.

# Global Constants

In [1]:
GDRIVE_DIR = "/content/drive"
GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"

#  Import Python packages

In [2]:
import pandas as pd
import functools

from google.colab import drive

#  Define metrics and parameters

In this section we are going to define the parameters used to collect the data and the metrics used. I will consider Bitcoin data for 10 years, starting from 2012-01-01 through 2022-12-31.

Note that since the Blockchain.com API allows retreiving data with a maximum timespan equal to 6 years, I manually computed the continue date so that I could make a second API call to get the remaining data.

Regarding the metrics, I chose the ones that seemed most relevant to me, containing both price statistics but also technical features of Bitcoin's blockchain.

In [3]:
# Define the parameters
timespan = "6years" # Duration of the data (because the Max timespan == 6years)
start_date = "2012-01-01"
continue_date = "2017-12-31" # The continue date (manually calculate the continue_date)
end_date = "2022-12-31"

# Metrics considered
metrics = [
          ##Currency Statistics##
            "market-price", # Market Price: The average USD market price across major bitcoin exchanges.
            "trade-volume", #E xchange Trade Volume (USD): The total USD value of trading volume on major bitcoin exchanges.

          ##Block Details##
            "blocks-size", # Blockchain Size (MB): The total size of the blockchain minus database indexes in megabytes.
            "avg-block-size", # Average Block Size (MB): The average block size over the past 24 hours in megabytes.
            "n-transactions-total", # Total Number of Transactions: The total number of transactions on the blockchain.
            "n-transactions-per-block", # Average Transactions Per Block: The average number of transactions per block over the past 24 hours.

          ##Mining Information##
            "hash-rate", # Total Hash Rate (TH/s): The estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
            "difficulty", # Network Difficulty: A relative measure of how difficult it is to mine a new block for the blockchain.
            "miners-revenue", # Miners Revenue (USD): Total value of coinbase block rewards and transaction fees paid to miners.
            "transaction-fees-usd", # Total Transaction Fees (USD): The total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

          ##Network Activity##
            "n-unique-addresses", # The total number of unique addresses used on the blockchain.
            "n-transactions", # Confirmed Transactions Per Day: The total number of confirmed transactions per day.
            "estimated-transaction-volume-usd" # Estimated Transaction Value (USD): The total estimated value in USD of transactions on the blockchain. This does not include coins returned as change.
]

# Retreiving data

In this section we are going to make the call to the Blockchain.com API to retrieve the data.

In [4]:
def data_crawler(timespan, metrics, start_date, continue_date, end_date):
    # API Info
    url1 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'
    url2 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={continue_date}&format=csv'

    # Obtain Data
    data1 = pd.read_csv(url1,names=['timestamp',metrics])
    data2 = pd.read_csv(url2,names=['timestamp',metrics])

    # Concat by rows
    all_data = pd.concat([data1,data2])

    # Transform "timestamp" to datetime type
    all_data['timestamp'] = pd.to_datetime(all_data["timestamp"])

    # Keep the same end date with Bitcoin data
    all_data = all_data[(all_data['timestamp'] < end_date)]

    return all_data

In [5]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain Blockchain Data from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2012-01-01,5.04,0.000000e+00,861.941752,0.017073,2119853.0,32.686275,8.591401e+00,1.159929e+06,4.260652e+04,18.516384,8531.0,5001.0,1.016110e+06
1,2012-01-02,5.27,0.000000e+00,864.547504,0.019121,2124845.0,35.827815,8.764382e+00,1.159929e+06,6.301249e+04,35.989325,8928.0,5410.0,7.508830e+05
2,2012-01-03,5.45,0.000000e+00,867.445999,0.018212,2130220.0,36.308176,9.340986e+00,1.159929e+06,4.662806e+04,30.560129,9528.0,5773.0,6.037982e+05
3,2012-01-04,5.37,0.000000e+00,870.374487,0.019351,2135991.0,38.463087,8.879703e+00,1.159929e+06,4.706558e+04,78.082768,9542.0,5731.0,7.495462e+05
4,2012-01-05,5.80,0.000000e+00,873.246150,0.024677,2141802.0,47.578231,8.476080e+00,1.159929e+06,5.369470e+04,44.697203,11636.0,6994.0,1.614569e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4000,2022-12-26,16826.86,2.726254e+07,445014.991139,1.119264,790945257.0,1732.589147,2.267765e+08,3.536407e+13,1.410410e+07,296889.796293,548937.0,223504.0,8.333855e+08
4001,2022-12-27,16915.70,3.983614e+07,445159.265994,1.376405,791168987.0,2081.382609,2.021651e+08,3.536407e+13,1.248977e+07,400541.066884,593337.0,239359.0,1.240817e+09
4002,2022-12-28,16699.00,8.334204e+07,445318.173024,1.026325,791410511.0,1624.696429,2.953368e+08,3.536407e+13,1.784473e+07,344406.479474,661225.0,272949.0,1.926197e+09
4003,2022-12-29,16539.28,9.358288e+07,445490.189205,1.263212,791683136.0,1886.205674,2.478720e+08,3.536407e+13,1.497709e+07,341839.738394,668338.0,265955.0,2.178953e+09


In [6]:
# Check duplicated rows
len(df1['timestamp'].unique())

4005

Due to a problem with the Blockchain.com API, I was forced to make an additional call to retrieve capitalization and total circulating data that will be added to the currency statistics to get a single dataset.

In [7]:
# Retrieving market capitalization and total circulating data
metrics = [
  "total-bitcoins", # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
  "market-cap", # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

merge = functools.partial(pd.merge, on='timestamp')
df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2012-01-01 00:00:01,8001900.00,4.032958e+07
1,2012-01-02 13:34:31,8013350.00,4.223035e+07
2,2012-01-04 00:14:03,8025100.00,4.309479e+07
3,2012-01-05 15:23:53,8036850.00,4.661373e+07
4,2012-01-06 23:04:03,8048100.00,5.311746e+07
...,...,...,...
2843,2022-12-25 05:40:07,19242637.50,3.241037e+11
2844,2022-12-26 19:25:14,19243875.00,3.240861e+11
2845,2022-12-28 08:27:09,19245118.75,3.205852e+11
2846,2022-12-29 15:06:12,19246356.25,3.203364e+11


In [8]:
# Check duplicated rows
len(df2['timestamp'].unique())

2848

In [9]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)

In [10]:
df2

,timestamp,total-bitcoins,market-cap
0,2012-01-01,8001900.00,4.032958e+07
1,2012-01-02,8013350.00,4.223035e+07
2,2012-01-04,8025100.00,4.309479e+07
3,2012-01-05,8036850.00,4.661373e+07
4,2012-01-06,8048100.00,5.311746e+07
...,...,...,...
2843,2022-12-25,19242637.50,3.241037e+11
2844,2022-12-26,19243875.00,3.240861e+11
2845,2022-12-28,19245118.75,3.205852e+11
2846,2022-12-29,19246356.25,3.203364e+11


In [11]:
# Check duplicated rows
len(df2['timestamp'].unique())

2847

In [12]:
# Add the market capitalization and total circulating data
all_data = pd.merge(df1, df2, how="right", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2012-01-01,5.04,0.000000e+00,861.941752,0.017073,2119853.0,32.686275,8.591401e+00,1.159929e+06,4.260652e+04,18.516384,8531.0,5001.0,1.016110e+06,8001900.00,4.032958e+07
1,2012-01-02,5.27,0.000000e+00,864.547504,0.019121,2124845.0,35.827815,8.764382e+00,1.159929e+06,6.301249e+04,35.989325,8928.0,5410.0,7.508830e+05,8013350.00,4.223035e+07
2,2012-01-04,5.37,0.000000e+00,870.374487,0.019351,2135991.0,38.463087,8.879703e+00,1.159929e+06,4.706558e+04,78.082768,9542.0,5731.0,7.495462e+05,8025100.00,4.309479e+07
3,2012-01-05,5.80,0.000000e+00,873.246150,0.024677,2141802.0,47.578231,8.476080e+00,1.159929e+06,5.369470e+04,44.697203,11636.0,6994.0,1.614569e+06,8036850.00,4.661373e+07
4,2012-01-06,6.60,0.000000e+00,876.852128,0.021753,2148716.0,38.853801,9.802269e+00,1.159929e+06,6.043696e+04,46.957274,11057.0,6644.0,1.155595e+06,8048100.00,5.311746e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2842,2022-12-25,16838.10,3.633847e+07,444875.815742,0.987937,790740923.0,1456.127660,2.478720e+08,3.536407e+13,1.507374e+07,230836.689777,522307.0,205314.0,5.677043e+08,19242637.50,3.241037e+11
2843,2022-12-26,16826.86,2.726254e+07,445014.991139,1.119264,790945257.0,1732.589147,2.267765e+08,3.536407e+13,1.410410e+07,296889.796293,548937.0,223504.0,8.333855e+08,19243875.00,3.240861e+11
2844,2022-12-28,16699.00,8.334204e+07,445318.173024,1.026325,791410511.0,1624.696429,2.953368e+08,3.536407e+13,1.784473e+07,344406.479474,661225.0,272949.0,1.926197e+09,19245118.75,3.205852e+11
2845,2022-12-29,16539.28,9.358288e+07,445490.189205,1.263212,791683136.0,1886.205674,2.478720e+08,3.536407e+13,1.497709e+07,341839.738394,668338.0,265955.0,2.178953e+09,19246356.25,3.203364e+11


In [13]:
# Check nan value
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [14]:
# Check duplicated rows
len(all_data['timestamp'].unique())

2847

In [15]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2012-01-01,5.04,0.000000e+00,861.941752,0.017073,2119853.0,32.686275,8.591401e+00,1.159929e+06,4.260652e+04,18.516384,8531.0,5001.0,1.016110e+06,8001900.00,4.032958e+07
1,2012-01-02,5.27,0.000000e+00,864.547504,0.019121,2124845.0,35.827815,8.764382e+00,1.159929e+06,6.301249e+04,35.989325,8928.0,5410.0,7.508830e+05,8013350.00,4.223035e+07
2,2012-01-04,5.37,0.000000e+00,870.374487,0.019351,2135991.0,38.463087,8.879703e+00,1.159929e+06,4.706558e+04,78.082768,9542.0,5731.0,7.495462e+05,8025100.00,4.309479e+07
3,2012-01-05,5.80,0.000000e+00,873.246150,0.024677,2141802.0,47.578231,8.476080e+00,1.159929e+06,5.369470e+04,44.697203,11636.0,6994.0,1.614569e+06,8036850.00,4.661373e+07
4,2012-01-06,6.60,0.000000e+00,876.852128,0.021753,2148716.0,38.853801,9.802269e+00,1.159929e+06,6.043696e+04,46.957274,11057.0,6644.0,1.155595e+06,8048100.00,5.311746e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2842,2022-12-25,16838.10,3.633847e+07,444875.815742,0.987937,790740923.0,1456.127660,2.478720e+08,3.536407e+13,1.507374e+07,230836.689777,522307.0,205314.0,5.677043e+08,19242637.50,3.241037e+11
2843,2022-12-26,16826.86,2.726254e+07,445014.991139,1.119264,790945257.0,1732.589147,2.267765e+08,3.536407e+13,1.410410e+07,296889.796293,548937.0,223504.0,8.333855e+08,19243875.00,3.240861e+11
2844,2022-12-28,16699.00,8.334204e+07,445318.173024,1.026325,791410511.0,1624.696429,2.953368e+08,3.536407e+13,1.784473e+07,344406.479474,661225.0,272949.0,1.926197e+09,19245118.75,3.205852e+11
2845,2022-12-29,16539.28,9.358288e+07,445490.189205,1.263212,791683136.0,1886.205674,2.478720e+08,3.536407e+13,1.497709e+07,341839.738394,668338.0,265955.0,2.178953e+09,19246356.25,3.203364e+11


In [16]:
def move_columns(dataset, target_colum, column_to_move):
  cols = list(dataset.columns)
  cols.remove(column_to_move)
  cols.insert(cols.index(target_colum)+1, column_to_move)
  dataset = dataset.reindex(columns=cols)

  return dataset

In [17]:
# Move the column 'total-bitcoins' and 'market-cap' after the column 'market-price'
all_data = move_columns(all_data, 'market-price', 'total-bitcoins')
all_data = move_columns(all_data, 'market-price', 'market-cap')
all_data

,timestamp,market-price,market-cap,total-bitcoins,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2012-01-01,5.04,4.032958e+07,8001900.00,0.000000e+00,861.941752,0.017073,2119853.0,32.686275,8.591401e+00,1.159929e+06,4.260652e+04,18.516384,8531.0,5001.0,1.016110e+06
1,2012-01-02,5.27,4.223035e+07,8013350.00,0.000000e+00,864.547504,0.019121,2124845.0,35.827815,8.764382e+00,1.159929e+06,6.301249e+04,35.989325,8928.0,5410.0,7.508830e+05
2,2012-01-04,5.37,4.309479e+07,8025100.00,0.000000e+00,870.374487,0.019351,2135991.0,38.463087,8.879703e+00,1.159929e+06,4.706558e+04,78.082768,9542.0,5731.0,7.495462e+05
3,2012-01-05,5.80,4.661373e+07,8036850.00,0.000000e+00,873.246150,0.024677,2141802.0,47.578231,8.476080e+00,1.159929e+06,5.369470e+04,44.697203,11636.0,6994.0,1.614569e+06
4,2012-01-06,6.60,5.311746e+07,8048100.00,0.000000e+00,876.852128,0.021753,2148716.0,38.853801,9.802269e+00,1.159929e+06,6.043696e+04,46.957274,11057.0,6644.0,1.155595e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2842,2022-12-25,16838.10,3.241037e+11,19242637.50,3.633847e+07,444875.815742,0.987937,790740923.0,1456.127660,2.478720e+08,3.536407e+13,1.507374e+07,230836.689777,522307.0,205314.0,5.677043e+08
2843,2022-12-26,16826.86,3.240861e+11,19243875.00,2.726254e+07,445014.991139,1.119264,790945257.0,1732.589147,2.267765e+08,3.536407e+13,1.410410e+07,296889.796293,548937.0,223504.0,8.333855e+08
2844,2022-12-28,16699.00,3.205852e+11,19245118.75,8.334204e+07,445318.173024,1.026325,791410511.0,1624.696429,2.953368e+08,3.536407e+13,1.784473e+07,344406.479474,661225.0,272949.0,1.926197e+09
2845,2022-12-29,16539.28,3.203364e+11,19246356.25,9.358288e+07,445490.189205,1.263212,791683136.0,1886.205674,2.478720e+08,3.536407e+13,1.497709e+07,341839.738394,668338.0,265955.0,2.178953e+09


Once we have the daily dataset we will go to sample it at a frequency of 1 minute (1T) using the resample method. This means that the data will be organized in 1-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the DataFrame by estimating missing values based on the surrounding known values.

In [18]:
# Upsampling to 1min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_1m = all_data.resample('1T').interpolate()
all_data_1m

,market-price,market-cap,total-bitcoins,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2012-01-01 00:00:00,5.040000,4.032958e+07,8.001900e+06,0.000000e+00,861.941752,0.017073,2.119853e+06,32.686275,8.591401e+00,1.159929e+06,4.260652e+04,18.516384,8531.000000,5001.000000,1.016110e+06
2012-01-01 00:01:00,5.040160,4.033090e+07,8.001908e+06,0.000000e+00,861.943562,0.017074,2.119856e+06,32.688456,8.591521e+00,1.159929e+06,4.262069e+04,18.528518,8531.275694,5001.284028,1.015925e+06
2012-01-01 00:02:00,5.040319,4.033222e+07,8.001916e+06,0.000000e+00,861.945371,0.017076,2.119860e+06,32.690638,8.591641e+00,1.159929e+06,4.263486e+04,18.540652,8531.551389,5001.568056,1.015741e+06
2012-01-01 00:03:00,5.040479,4.033354e+07,8.001924e+06,0.000000e+00,861.947181,0.017077,2.119863e+06,32.692819,8.591761e+00,1.159929e+06,4.264903e+04,18.552786,8531.827083,5001.852083,1.015557e+06
2012-01-01 00:04:00,5.040639,4.033486e+07,8.001932e+06,0.000000e+00,861.948990,0.017079,2.119867e+06,32.695001,8.591881e+00,1.159929e+06,4.266320e+04,18.564920,8532.102778,5002.136111,1.015373e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-29 23:56:00,16630.236639,3.195315e+11,1.924759e+07,6.395011e+07,445667.774833,1.356171,7.919473e+08,1841.990251,2.776742e+08,3.536407e+13,1.671052e+07,360654.553672,725964.480556,290945.388889,3.107253e+09
2022-12-29 23:57:00,16630.299979,3.195310e+11,1.924759e+07,6.392947e+07,445667.898499,1.356236,7.919475e+08,1841.959461,2.776950e+08,3.536407e+13,1.671173e+07,360667.655911,726004.610417,290962.791667,3.107900e+09
2022-12-29 23:58:00,16630.363319,3.195304e+11,1.924759e+07,6.390884e+07,445668.022166,1.356300,7.919477e+08,1841.928670,2.777157e+08,3.536407e+13,1.671294e+07,360680.758150,726044.740278,290980.194444,3.108546e+09


# Output

In this last section we are going to save the dataset we just created to the Google Drive.

In [ ]:
# Link Colab to our Google Drive
drive.mount(GDRIVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def output(dataset, path):
  dataset.to_parquet(path)

In [ ]:
# Output the 1d data
GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_1d"
GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT
output(all_data, GDRIVE_DATASET)

In [ ]:
# Output the 1m data
GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_1m"
GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT
output(all_data_1m, GDRIVE_DATASET)